# TruePairVsSkype_21dpf


In [ ]:
%config InteractiveShellApp.pylab_import_all = False
%matplotlib inline
%pylab inline
%reload_ext autoreload
%autoreload 2

import sys
import os
import fnmatch

import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame, Series
import seaborn as sns
import glob
from datetime import datetime
from scipy import stats

if 'startDirMaster' not in locals():
    startDirMaster=os.getcwd()

propsFn=startDirMaster+'\\props.csv'

props=pd.read_csv(propsFn, header=None, index_col=0, squeeze=True,delim_whitespace=True).to_dict()

base=props['BaseDir']
expFile=props['allExpFn']

RawDataDir = os.path.join(base,props['RawDataDir'])+'\\'
ProcessingDir = os.path.join(base,props['ProcessingDir'])+'\\Fig1TruePairVsSkype\\'
outputDir = os.path.join(base,props['outputDir'])+'\\'

if not os.path.isdir(ProcessingDir):
    os.makedirs(ProcessingDir)
if not os.path.isdir(outputDir):
    os.makedirs(outputDir)

os.chdir('..\\')
import functions.matrixUtilities_joh as mu
import matplotlib.pyplot as plt
import models.experiment as xp
import models.experiment_set as es
import functions.paperFigureProps as pfp
pfp.paper()
inToCm=2.54


# Select experiments for batch loading and processing
## 1) Load experiments of virtual interactions

In [ ]:
# Get experiments tagged with StimulusProtocol == 9
infoAll=pd.read_csv(expFile, sep=',')
infoAll=infoAll[infoAll.stimulusProtocol=='9']
infoAll.head()

Pre-Analyze all experiments only if necessary, this takes a couple of minutes! Experiment summary csv files are saved to disk.

In [ ]:
# collect meta information and save to new csv file for batch processing

aviPath=[]
posPath=[]
PLPath=[]
expTime = []
    
for index,row in infoAll.iterrows():
    startDir=RawDataDir+row.path+'\\'
    #startDir='D:\\data\\b\\2017\\'+row.path+'\\'
    #if not os.path.isdir(startDir):
    #    startDir='E:\\b\\2017\\'+row.path+'\\'
        
    posPath.append(glob.glob(startDir+'PositionTxt*.txt')[0])
    PLPath.append(glob.glob(startDir+'PL*.txt')[0])
    
    head, tail = os.path.split(posPath[-1])
    currTime=datetime.strptime(tail[-23:-4], '%Y-%m-%dT%H_%M_%S')
    expTime.append(currTime)
    
infoAll['txtPath']=posPath
infoAll['pairList']=PLPath

infoAll['epiDur'] = 5      # duration of individual episodes (default: 5 minutes)
infoAll['episodes'] = -1   # number of episodes to process: -1 to load all episodes (default: -1)
infoAll['inDish'] = 10#np.arange(len(posPath))*120     # time in dish before experiments started (default: 10)
infoAll['arenaDiameter_mm'] = 100 # arena diameter (default: 100 mm)
infoAll['minShift'] = 60 # minimum number of seconds to shift for control IAD
infoAll['episodePLcode'] = 0 # flag if first two characters of episode name encode animal pair matrix (default: 0)
infoAll['recomputeAnimalSize'] = 1 # flag to compute animals size from avi file (takes time, default: 1)
infoAll['SaveNeighborhoodMaps'] = 1 # flag to save neighborhood maps for subsequent analysis (takes time, default: 1)
infoAll['computeLeadership'] = 0 # flag to compute leadership index (takes time, default: 1)
infoAll['ComputeBouts'] = 1 # flag to compute swim bout frequency (takes time, default: 1)
infoAll['set'] = np.arange(len(posPath))   # experiment set: can label groups of experiments (default: 0)
infoAll['ProcessingDir']=ProcessingDir
infoAll['outputDir']=outputDir

infoAll['expTime']=expTime
    
infoAll.head()

In [ ]:
# Get experiments tagged with StimulusProtocol == 8
info=pd.read_csv(expFile, sep=',')
info=info[info.stimulusProtocol=='8']
info.head()

In [ ]:
posPath = []
aviPath=[]
bdGroupAll=[]
bgAll=[]
expTimeAll=[]

for index,row in info.iterrows():
    DishDir=RawDataDir+row.path
    #DishDir='D:\\data\\b\\2017\\'+row.path
    currTime=datetime.strptime(row.date, '%Y%m%d%H%M%S')
    #if not os.path.isdir(DishDir):
    #    DishDir='E:\\b\\2017\\'+row.path
    for root, dirnames, filenames in os.walk(DishDir):
        for filename in fnmatch.filter(filenames, '*nogaps*.txt'):
            posPath.append(os.path.join(root, filename))
            bdGroupAll.append('0 0')
            bgAll.append(row.bd)
            expTimeAll.append(currTime)
        for filename in fnmatch.filter(filenames, '*split*.avi'):
            aviPath.append(os.path.join(root, filename))


info=pd.DataFrame({'txtPath': posPath})
info['epiDur']=5
info['bd']=bgAll
info['bdGroup']=bdGroupAll
info['episodes']=-1
info['inDish']=10
info['stimulusProtocol']=8
info['arenaDiameter_mm'] = 100 # arena diameter (default: 100 mm)
info['minShift'] = 60 # minimum number of seconds to shift for control IAD
info['episodePLcode'] = 0 # flag if first two characters of episode name encode animal pair matrix (default: 0)
info['recomputeAnimalSize'] = 1 # flag to compute animals size from avi file (takes time, default: 1)
info['SaveNeighborhoodMaps'] = 1 # flag to save neighborhood maps for subsequent analysis (takes time, default: 1)
info['computeLeadership'] = 0 # flag to compute leadership index (takes time, default: 1)
info['ComputeBouts'] = 1 # flag to compute swim bout frequency (takes time, default: 1)
info['set'] = np.arange(len(posPath))   # experiment set: can label groups of experiments (default: 0)
info['expTime']=expTimeAll
#info['birthDayAll']=birthDay_all
info['ProcessingDir']=ProcessingDir
info['outputDir']=outputDir

info.head()

In [ ]:
infoVirtualPhysical=pd.concat([info,infoAll],sort=True)

csvFile=os.path.join(ProcessingDir,'Fig1_VirtualVsPhysical.csv')
infoVirtualPhysical.to_csv(csvFile,encoding='utf-8')
infoVirtualPhysical.head()

In [ ]:

[expPath,expID,expInv,expCount]=np.unique([mu.splitall(x[x.find('RawData'):])[1] for x in infoVirtualPhysical.txtPath],
                          return_index=True, 
                          return_inverse=True,
                          return_counts = True)

print(expPath)
print(expID)
print(expInv)
print(expCount)

In [ ]:
def CountConsecutives(x):
    InGroupID=[0]
    i=0
    last=x[0]
    for g in x[1:]:

        if last==g:
            i+=1
            InGroupID.append(i)

        else:
            InGroupID.append(0)
            i=0
        last=g
    return np.array(InGroupID)   

InGroupID=CountConsecutives(expInv)
print(InGroupID)

In [ ]:


def readExperiment(keepData=False):
    tmp=es.experiment_set(csvFile=csvFile)
    if keepData:
        return tmp
    else:
        return 1

expSet=readExperiment(keepData=True)


In [ ]:
useAnimalsSI=np.array([0,2,4,6,8,12,14])
idx=(df.age==26)

IADsAll=[]
IADsAnAll=[]
expList=df[idx].animalSet.unique()
for e in expList:
    dfTmp=df[df.animalSet==e]
    idx2=dfTmp.animalIndexMatch.isin(useAnimalsSI)
    
    print((np.abs(np.diff(expSet.experiments[e].shiftList,axis=0))/30).astype('int'))

    anList=dfTmp[idx2].animalIndexMatch.unique()
    
    for a in anList:
        idx3=dfTmp.animalIndexMatch==a
        pl=np.where(idx2&idx3)[0]

        anIAD=[]
        for p in pl:
            expIAD=np.array(expSet.experiments[e].pair[p].IADs())
            expIADm=np.nanmean(expIAD,axis=1)
            anIAD.append(expIADm)
            for i in range(9):
                ShiftAttract=(expIADm[-1]-expIADm[i])/expIADm[-1]
                IADsAll.append(ShiftAttract)
        anIAD=np.array(anIAD).mean(axis=0)
        for i in range(9):
            ShiftAttract=(anIAD[-1]-anIAD[i])/anIAD[-1]
            IADsAnAll.append(ShiftAttract)

IADsAll=np.array(IADsAll)
IADsAnAll=np.array(IADsAnAll)

In [ ]:
IADsAnAll.shape[0]/9

In [ ]:
expList

In [ ]:
h=plt.hist(IADsAll,bins=100)

In [ ]:
plt.hist(IADsAnAll,bins=h[1]);
print(2*np.std(IADsAnAll))

In [ ]:
a=IADsAll
print(2*np.std(a))
print(stats.t.interval(0.95, len(a)-1, loc=np.mean(a), scale=np.std(a)/np.sqrt(6)))

In [ ]:
stats.norm.interval(0.95, loc=np.mean(a), scale=np.std(a))

In [ ]:
csvPath = []
for f in [mu.splitall(x)[-1][:-4] for x in infoVirtualPhysical.txtPath]:
    csvPath.append(glob.glob(ProcessingDir+f+'*siSummary*.csv')[0])


In [ ]:

df=pd.DataFrame()
i=0
for fn in csvPath:
    print(fn)
    tmp=pd.read_csv(fn,index_col=0,sep=',')
    tmp.animalSet=i
    df=pd.concat([df,tmp])
    i+=1
df['episode']=0
print('df shape',df.shape)

In [ ]:
d=df.time
r=datetime(2017,1,1)
t2=[pd.to_datetime(x).replace(day=1,month=1)for x in df.time]
t3=[(x-r)/pd.Timedelta('1 hour') for x in t2]
t4=[pd.to_datetime(x).replace(hour=1,minute=1,second=1)for x in df.time]
#df['t2']=t2
df['timeHH']=t3
df['timeDD']=t4

DishOrder=np.array([5,6,7,8,1,2,3,4,13,14,15,16,9,10,11,12])-1

df['animalIndexMatch']=DishOrder[df['animalIndex']]
df['inGroupSetID']=InGroupID[df.animalSet]
df.loc[df.stimulusProtocol==8,'animalIndexMatch']=df.loc[df.stimulusProtocol==8,'animalIndex']
df.loc[df.stimulusProtocol==8,'animalIndexMatch']=df.loc[df.stimulusProtocol==8,'animalIndex']+(2*df.loc[df.stimulusProtocol==8,'inGroupSetID'])
df['animalSetCorrect']=expInv[df.animalSet]

df['animalIDRun']=df.animalIndexMatch+16*df.animalSetCorrect
df.head()


In [ ]:
df

In [ ]:
minAge=17
tmp=df[(df.animalIndexMatch.isin(useAnimalsSI))&(df.age>=minAge)]
SI=tmp.groupby(['animalIDRun','stimulusProtocol'])['si','age'].mean().reset_index()
dfPlt=SI.pivot_table(index=['age','animalIDRun'],columns='stimulusProtocol',values=['si']).reset_index()

pfp.paper()
sns.set_palette('viridis',4)
inToCm=2.54
plt.figure(figsize=(4.5/inToCm,4.5/inToCm))
ax = plt.gca()

groups = dfPlt.groupby('age')
for name, group in groups:
    ax.plot(group.si[8], group.si[9], marker='o', linestyle='', ms=5, label=name)

ax.set_ylim([-.1,1])
ax.set_xlim([0,1])
ax.set_yticks([0,.5,1]);
ax.set_xticks([0,.5,1]);
ax.set_xlabel('Physical attraction')
ax.set_ylabel('Virtual attraction')
plt.axhline(0,ls=':',color='k')
sns.despine()

s,i,r,p,std=stats.linregress(dfPlt.si[8],dfPlt.si[9])
t=np.linspace(0,1,100)
l=i+s*t
ax.plot(t,l,ls=':',color='gray')
ax.text(.1,.9,'R: '+str(r)[:4])
ax.text(.1,.8,'p: '+str(p)[:3]+str(p)[-4:])
figPath=outputDir+'\\1C_physicalVsVirtual_SI_correlation.svg'
plt.savefig(figPath)

In [ ]:
#iadTrace of animals 0-1 on day 10_13 virtual
idxV=(infoVirtualPhysical.stimulusProtocol=='9') & (expInv == 0)
selectExpV = expSet.experiments[np.where(idxV)[0][0]]
idxSubV = np.where(df[df.animalSet==np.where(idxV)[0][0]].animalIndex==0)[0]
iadV=np.concatenate([selectExpV.pair[x].IAD() for x in idxSubV])

idxP=(infoVirtualPhysical.stimulusProtocol==8) & (expInv == 0) & (InGroupID == 0)
selectExpP = expSet.experiments[np.where(idxP)[0][0]]
idxSubP = np.where(df[df.animalSet==np.where(idxP)[0][0]].animalIndex==0)[0]
iadP=np.concatenate([selectExpP.pair[x].IAD() for x in idxSubP])

iad_mV=np.mean([selectExpV.pair[x].IAD_m() for x in idxSubV])
iad_mP=np.mean([selectExpP.pair[x].IAD_m() for x in idxSubP])
spiad_mV=np.mean([selectExpV.pair[x].spIAD_m() for x in idxSubV])
spiad_mP=np.mean([selectExpP.pair[x].spIAD_m() for x in idxSubP])

iad_mV2=[iad_mV,spiad_mV]
iad_mP2=[iad_mP,spiad_mP]

pdSiBoth=dfPlt[dfPlt.age==26].reset_index()
pdSiBoth.columns.set_levels(['physical','virtual',''],level=1,inplace=True)
pdSiBoth=pdSiBoth.si.unstack().reset_index()
pdSiBoth.columns=['condition','level','si']
pdSiBoth

In [ ]:
pfp.paper()

import matplotlib.gridspec as gridspec
IAD_plotTime=15
IAD_plotStart=50
IAD_plot_rng=np.arange(IAD_plotStart*30*60,(IAD_plotStart+IAD_plotTime)*30*60)
plt_subsample=15
fig = plt.figure()
gs = gridspec.GridSpec(1, 5, width_ratios=[4,1,4,1,1.5]) 
ax = fig.add_subplot(gs[0])
axes = [ax] + [fig.add_subplot(gs[i], sharey=ax) for i in range(1, 4)]

iad=iadP[IAD_plot_rng][::plt_subsample]
xax=np.linspace(0,20,iad.shape[0])
axes[0].plot(xax,iad,color='gray',ms=2,marker='.',ls='')

iad_m=iad_mP2
      
barlist=axes[1].bar([1,2],iad_m,edgecolor=['gray','gray'],color=['gray','w'],linewidth=1)

#axes[1].errorbar([2.5],iad_m[1],yerr=iad_sd,ls='',color='k',capthick=0)
axes[0].axhline(iad_m[1],ls=':',color='gray')
xTickLocation=[1,2]

iad=iadV[IAD_plot_rng][::plt_subsample]
axes[2].plot(xax,iad,'k.',ms=2)

iad_m=iad_mV2

barlist=axes[3].bar([1,2],iad_m,edgecolor=['k','k'],color=['k','w'],linewidth=1)

#axes[3].errorbar([2.5],iad_m[1],yerr=iad_sd, ls='',color='k',capthick=0)
axes[2].axhline(iad_m[1],ls=':',color='gray')

axes[0].set_xlabel('Time (minutes)')
axes[2].set_xlabel('Time (minutes)')
axes[0].set_ylabel('IAD: Inter animal \n distance (mm)',labelpad=1)
axes[0].set_title('Physical interaction')
axes[2].set_title('Virtual interaction')
axes.append(fig.add_subplot(gs[4]))

col2=[[.5,.5,.5],'k']


sns.pointplot(x="condition", y="si", data=pdSiBoth,ci='sd',zorder=50,join=False,palette=['k'],errwidth=1,capsize=.3)
sns.pointplot(x="condition", y="si", data=pdSiBoth,ci=None,zorder=100,join=False,palette=['r'],markers=["_"],scale=3)

sns.swarmplot(data=pdSiBoth,ax=axes[4],x='condition',y='si',zorder=-1,palette=col2)

axes[4].set_ylim([0,1])
#axes[4].set_xlim([-.5,1.5])
axes[4].set_xlabel('')
axes[4].set_ylabel('Attraction \n (Shuffled-Real)/Shuffled')
axes[4].set_xticklabels(['Physical','Virtual'])

for tick in axes[4].xaxis.get_major_ticks():
    #tick.label.set_fontsize(12) 
    tick.label.set_rotation(45)
    
    
for ax in [axes[x]for x in [1,3]]:
    ax.set_ylabel('')
    ax.set_xlim([0,3])
    ax.set_xticks(xTickLocation)
    ax.set_xticklabels(['Real','Shuffled']);
    ax.tick_params(axis='x', which='both',length=0)
    for tick in ax.xaxis.get_major_ticks():
        tick.label.set_rotation(45)

axes[0].set_ylim([0,100])
axes[0].yaxis.tick_left()
axes[0].spines['top'].set_color('white')
axes[0].spines['right'].set_color('white')


axes[4].yaxis.tick_right()
axes[4].yaxis.set_label_position("right")
axes[4].spines['top'].set_color('white')
axes[4].spines['left'].set_color('white')

for ax in axes[1:4]:
    plt.setp(ax.get_yticklabels(), visible=False)
    ax.spines['left'].set_color('white')
    ax.spines['top'].set_color('white')
    ax.spines['right'].set_color('white')
    ax.tick_params(axis='y', which='both',length=0)
    ax.tick_params(axis='x', which='top',length=0)

for ax in axes:
    ax.xaxis.set_ticks_position('bottom')

plt.tight_layout(pad=0, w_pad=0, h_pad=0);
figPath=outputDir+'\\1B_physicalVsVirtual_IAD_SI.svg'
plt.savefig(figPath,dpi=150)

from shutil import copy2

def splitall(path):
    allparts = []
    while 1:
        parts = os.path.split(path)
        if parts[0] == path:  # sentinel for absolute paths
            allparts.insert(0, parts[0])
            break
        elif parts[1] == path: # sentinel for relative paths
            allparts.insert(0, parts[1])
            break
        else:
            path = parts[0]
            allparts.insert(0, parts[1])
    return allparts


for i,row in info.iterrows():
    fn=row.txtPath
    head, tail = os.path.split(fn)

    copyList=[]
    copyList.append(glob.glob(head+'\\bgMed*.csv')[0])
    copyList.append(glob.glob(head+'\\*nogaps.txt')[0])
    copyList.append(glob.glob(head+'\\*anSize.csv')[0])
    


    for f in copyList:
        print(f)
        if f[0]=='E':
            keepSlash=3
        else:
            keepSlash=4
        toDirectory = "e:\\b\\LarschAndBaier2018\\RawData\\" + os.path.join(*splitall(f)[keepSlash:-1])+"\\"
        #toDirectory = "e:\\b\\LarschAndBaier2018\\RawData\\" 
        if not os.path.isdir(toDirectory):
            os.makedirs(toDirectory)
        
        copy2(f, toDirectory)
        #os.chdir(toDirectory)
        if 'nogaps.txt' in f:
            old=glob.glob(toDirectory+'\\*nogaps.txt')[0]
            t=datetime.strftime(row.expTime, '%Y-%m-%dT%H_%M_%S')
            new=old[:-4]+str(i).zfill(2)+"_"+t+'.txt'
            os.rename(old,new)
            print(new)


for i,row in infoAll.iterrows():
    fn=row.txtPath
    head, tail = os.path.split(fn)

    copyList=[]
    copyList.append(glob.glob(head+'\\ROI*.csv')[0])
    copyList.append(glob.glob(head+'\\PositionTxt*.txt')[0])
    copyList.append(glob.glob(head+'\\PL*.txt')[0])
    copyList.append(glob.glob(head+'\\*anSize.csv')[0])
    
    for f in copyList:
        print(f)
        if f[0]=='E':
            keepSlash=3
        else:
            keepSlash=4
        toDirectory = "e:\\b\\LarschAndBaier2018\\RawData\\" + os.path.join(*splitall(f)[keepSlash:-1])+"\\"
        #toDirectory = "e:\\b\\LarschAndBaier2018\\RawData\\" 
        if not os.path.isdir(toDirectory):
            os.makedirs(toDirectory)
        
        copy2(f, toDirectory)
        #os.chdir(toDirectory)
        if 'nogaps.txt' in f:
            old=glob.glob(toDirectory+'\\*nogaps.txt')[0]
            t=datetime.strftime(row.expTime, '%Y-%m-%dT%H_%M_%S')
            new=old[:-4]+str(i)+"_"+t+'.txt'
            os.rename(old,new)
            print(new)